In [1]:
import os
import nltk
from nltk import FreqDist
import pandas as pd
import matplotlib.pyplot as plt
import re
import requests
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_json('data.json')

In [3]:
df['name']

0         t-mobile-atlanta-13
1          t-mobile-atlanta-3
2         t-mobile-atlanta-39
3            t-mobile-austell
4          t-mobile-lilburn-2
5         t-mobile-atlanta-30
6         t-mobile-atlanta-23
7         t-mobile-atlanta-46
8     t-mobile-lithia-springs
9          t-mobile-atlanta-7
10        t-mobile-east-point
11        t-mobile-atlanta-35
12        t-mobile-atlanta-28
13      t-mobile-alpharetta-7
14        t-mobile-atlanta-43
Name: name, dtype: object

In [5]:
df.loc[df['name'] == 't-mobile-atlanta-30']['reviews']

5    [[I had a great experience switching from AT&T...
Name: reviews, dtype: object

In [6]:
subscription_key = 'c1005ffa18cc42ddbf916d2483bb2dd2'
assert subscription_key
text_analytics_base_url = "https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.0/"

In [7]:
def freq_words(x, terms = 30):
  all_words = ' '.join([text for text in x])
  all_words = all_words.split()
  
  fdist = FreqDist(all_words)
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})
  
  # selecting top 20 most frequent words
  d = words_df.nlargest(columns="count", n = terms) 
  plt.figure(figsize=(20,5))
  ax = sns.barplot(data=d, x= "word", y = "count")
  ax.set(ylabel = 'Count')
  plt.show()

In [8]:
for i in range(len(df)):
    
    idnum = 1
    docDic = []
    for rev in raw_revs:
        docDic.append({'id':idnum, 'language':'en', 'text': rev[0]})
        idnum += 1
        
    documents = {'documents' : docDic}
    key_phrase_api_url = text_analytics_base_url + "keyPhrases"
    headers   = {'Ocp-Apim-Subscription-Key': subscription_key}
    response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
    key_phrases = response.json()
    
    sentiment_api_url = text_analytics_base_url + "sentiment"
    response  = requests.post(sentiment_api_url, headers=headers, json=documents)
    sentiments = response.json()
    
    entity_linking_api_url = text_analytics_base_url + "entities"
    response  = requests.post(entity_linking_api_url, headers=headers, json=documents)
    entities = response.json()
    pprint(entities)
    
    keyphr = []
    senti = []
    try:
        for i in range(len(key_phrases['documents'])):
            keyphr += key_phrases['documents'][i]['keyPhrases']
            senti += [sentiments['documents'][i]['score']]
    except:
        pprint(key_phrases)
    
#     freq_words(keyphr)
    
    currStore = df.loc[i]['name']
    print("Store: " + currStore)
    print("Loc: " + str(i))
    raw_revs = df.loc[i]['reviews']
    scores = [rev[1] for rev in raw_revs]
    print ("Average Score: %f" % (sum(scores) / float(len(scores))))

#     if (sum(senti) / float(len(senti)) > 0.5):
#         print("HAPPY")
#     else:
#         print("BAD")
    

NameError: name 'raw_revs' is not defined